In [1]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [2]:
import galois
import numpy as np
import time
import os
from scipy import stats

In [3]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(8)

In [4]:
#definisco una matrice k x (n-k)
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [10]:
#questa è con l'esempio di 4.3
A = field([[6, 6, 3, 0],
           [7, 5, 0, 7],
           [5, 0, 6, 1]])

In [ ]:
A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(A)

In [11]:
matrix_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrix_bin1 = field(matrix_bin1)

matrix_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrix_bin2 = field(matrix_bin2)

In [12]:
A_primo = field(matrix_bin1 @ A @ matrix_bin2)
print(A_primo)

[[5 1 0 6]
 [6 0 6 3]
 [7 7 5 0]]


In [8]:
#number of iteration for Montecarlo simulatiom
num_iterazioni = 2500

In [14]:
# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix):
    vector_of_dict = []
    for row in matrix:
        dictionary = {i: 0 for i in range(8)}
        print("Dictionary: ", dictionary)
        for value in row:
            value_as_int = int(value)
            dictionary[value_as_int] += 1
        
        vector_of_dict.append(dictionary)
        print("Vector of dict: ", vector_of_dict)

    def sort_key(dictionary):
        # torna tupla con valori del dizionario da 0 a 126
        return tuple(dictionary[i] for i in range(8))
    
    # Ordina la lista di dizionari usando la chiave personalizzata
    sorted_index = sorted(range(len(vector_of_dict)), key=lambda i: sort_key(vector_of_dict[i]), reverse=True)
    
    return matrix[sorted_index]

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    return sorted_matrix, num_swap_columns

def algorithm_test(matrix1, matrix2):
    row1_sort = order_row(matrix1)
    print(row1_sort)
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    print(matrix1_sorted)

    row2_sort = order_row(matrix2)
    print(row2_sort)
    matrix2_sorted = sort_by_column(row2_sort, range(k), 0)
    print(matrix2_sorted)

    #[0] because retur an array with 2 elements
    return np.array_equal(matrix1_sorted[0], matrix2_sorted[0])



In [15]:
algorithm_test(A, A_primo)

Dictionary:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
Vector of dict:  [{0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 2, 7: 0}]
Dictionary:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
Vector of dict:  [{0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 2, 7: 0}, {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 2}]
Dictionary:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
Vector of dict:  [{0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 2, 7: 0}, {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 2}, {0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0}]
[[5 0 6 1]
 [6 6 3 0]
 [7 5 0 7]]
Funzione ordina per colonna
(GF([[0, 1, 5, 6],
    [6, 0, 6, 3],
    [5, 7, 7, 0]], order=2^3), 4)
Dictionary:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
Vector of dict:  [{0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0}]
Dictionary:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
Vector of dict:  [{0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0}, {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 2, 7: 0}]
Dictionary:  {0: 

True